In [1]:
import pandas as pd
import numpy as np

In [2]:
from src.PortfolioIndicator import PortfolioIndicator
from src.PortfolioConstructor import PortfolioConstructor

### Import Data

In [3]:
data_directory = "data/"
file = "yahoo_data_filter_0_1.csv"

In [4]:
df = pd.read_csv(data_directory+file)
df.index = pd.to_datetime(df.iloc[:,0])
df = df.iloc[:,1:]

In [5]:
df.head()

,ADA-USD,ADX-USD,AE-USD,ANT-USD,ARDR-USD,ARK-USD,ATB-USD,BAT-USD,BCH-USD,BLOCK-USD,...,XEM-USD,XLM-USD,XMR-USD,XRP-USD,XUC-USD,XVG-USD,XZC-USD,ZEC-USD,ZEN-USD,ZRX-USD
date,,,,,,,,,,,,,,,,,,,,,
2017-12-31,0.719695,2.78937,1.32136,3.88903,1.72013,7.04232,0.447756,0.412203,2533.010010,44.567799,...,1.03405,0.360756,349.026001,2.30057,8.88856,0.222600,112.322998,505.510986,59.787899,0.895721
2018-01-01,0.728657,3.10880,1.30022,4.27365,1.60770,7.60626,0.445190,0.475599,2432.540039,41.992901,...,1.04137,0.480008,358.989990,2.39103,9.01810,0.156131,125.442001,577.192993,56.268101,0.877714
2018-01-02,0.782587,2.95154,1.38313,4.60514,1.83907,7.87181,0.462033,0.563300,2711.000000,44.665798,...,1.16018,0.564766,383.416992,2.48090,9.47555,0.144905,126.078003,583.505981,52.121899,0.896672
2018-01-03,1.079660,2.85780,1.52833,6.55136,1.95408,7.59385,0.490322,0.651219,2608.689941,49.891701,...,1.79028,0.896227,412.062988,3.10537,9.92106,0.145221,124.279999,581.888977,50.143101,0.974472
2018-01-04,1.114120,2.85536,1.80678,6.27358,1.78215,7.11654,0.474837,0.634125,2430.179932,50.342300,...,1.64481,0.724050,403.048004,3.19663,10.46710,0.159184,117.002998,595.190002,55.009499,1.045260


### Strategy Test

In [7]:
universe_df = df

#### Calculate Multiple Signal

In [8]:
momentum_period = 3
waiting_time = 1

In [9]:
complete_ret_sub_df_3 = universe_df.pct_change(periods=momentum_period).shift(waiting_time).dropna()

In [11]:
momentum_period = 1
waiting_time = 1

In [12]:
complete_ret_sub_df_1 = universe_df.pct_change(periods=momentum_period).shift(waiting_time).dropna()

In [14]:
complete_ret_sub_df_long_3 = complete_ret_sub_df_3.stack()

complete_ret_sub_df_long_1 = complete_ret_sub_df_1.stack()



In [15]:
signal_df_3 = complete_ret_sub_df_long_3.index.to_frame()
signal_df_3.index = range(0,signal_df_3.shape[0])
signal_df_3["momentum_3"] = list(complete_ret_sub_df_long_3)
signal_df_3.columns=["date","ticker","momentum_3"]

In [17]:
signal_df_1 = complete_ret_sub_df_long_1.index.to_frame()
signal_df_1.index = range(0,signal_df_1.shape[0])
signal_df_1["momentum_1"] = list(complete_ret_sub_df_long_1)
signal_df_1.columns=["date","ticker","momentum_1"]

In [25]:
signal_merge_df = signal_df_3.merge(signal_df_1,on=["date","ticker"],how="left")

#### Create Double Sort Indicator

In [88]:
signal_1 = "momentum_1"
signal_2 = "momentum_3"
df = signal_merge_df

rebalance_date_series = list(set(signal_df_3["date"]))

number_1 = 5
number_2 = 2

In [135]:
rebalance_date = rebalance_date_series[0]

In [136]:
sub_df = df[df["date"] == rebalance_date]

In [137]:
indicator_1 = pd.qcut(sub_df[signal_1],q=number_1,labels=False,duplicates="drop")

In [138]:
indicator_df_1 = pd.concat([sub_df["date"],sub_df["ticker"],indicator_1],axis=1)
indicator_df_1.columns = ["date","ticker","indicator_1"]

In [139]:
sub_df = sub_df.merge(indicator_df_1,on=["date","ticker"],how="left")

In [140]:
sub_df.index = sub_df["ticker"]

In [141]:
def qcut(x):
    
    indicator_2 = pd.qcut(x[signal_2],q=number_2,labels=False,duplicates="drop")
    
    return indicator_2

In [142]:
indicator_2 = sub_df.groupby("indicator_1").apply(qcut)

In [143]:
indicator_2

indicator_1  ticker   
0            ADA-USD      1
             ANT-USD      1
             ARK-USD      1
             ATB-USD      0
             EDG-USD      0
                         ..
4            SYS-USD      0
             VERI-USD     1
             WAVES-USD    0
             WINGS-USD    1
             XEM-USD      0
Name: momentum_3, Length: 95, dtype: int64

In [144]:
index_df.index = index_df.iloc[:,1]

In [145]:
index_df = indicator_2.index.to_frame()
index_df["indicator_2"] = indicator_2

In [146]:
index_df.index = range(0,index_df.shape[0])

In [147]:
indicator_2 = index_df

In [149]:
sub_df.index = range(0,sub_df.shape[0])

In [151]:
sub_df = sub_df.merge(indicator_2,on=["indicator_1","ticker"],how="left")

In [153]:
indicator_df = sub_df[["date","ticker","indicator_1","indicator_2"]]

In [154]:
indicator_df

,date,ticker,indicator_1,indicator_2
0,2020-09-26,ADA-USD,0,1
1,2020-09-26,ADX-USD,4,1
2,2020-09-26,AE-USD,2,1
3,2020-09-26,ANT-USD,0,1
4,2020-09-26,ARDR-USD,1,0
...,...,...,...,...
90,2020-09-26,XVG-USD,0,0
91,2020-09-26,XZC-USD,0,1
92,2020-09-26,ZEC-USD,3,1
93,2020-09-26,ZEN-USD,1,1
